This script made it possible to record Observador's Podcasts. The sound would then be transformed to text, but the final results weren't accurate and the idea ended up being discarded

In [3]:
import time
import pyaudio
import wave

from bs4 import BeautifulSoup as bs4
import requests

from selenium import webdriver
from selenium.webdriver.common.keys import Keys

from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

import soundcard as sc
import soundfile as sf

# Initialize audio recording parameters
chunk = 1024
sample_format = pyaudio.paInt16
channels = 2
rate = 44100
output_filename = "output.wav"

In [4]:
# Initialize the audio stream
audio = pyaudio.PyAudio()

In [5]:
driver = webdriver.Chrome(ChromeDriverManager().install())

[WDM] - Downloading: 100%|██████████| 6.30M/6.30M [00:01<00:00, 4.10MB/s]


In [16]:
# Open the webpage
url = 'https://observador.pt/programas/fact-check-2/salazar-criou-o-servico-nacional-de-saude/'  # Replace with the URL of the webpage with podcasts
driver.get(url)

In [17]:
# After rejecting cookies and all that stuff
# Wait for the video to load and get its duration
wait = WebDriverWait(driver, 3)  # Adjust the timeout as needed
video_element = wait.until(EC.presence_of_element_located((By.XPATH, '//div[@class="picture-and-controls"]')))

In [18]:
# Find the button element and click to play the video
play_button = driver.find_element_by_xpath('//div[@class="mejs__button mejs__playpause-button mejs__play"]/button')
play_button.click()

time.sleep(1)

soup = bs4(driver.page_source, 'html.parser')
video_duration_element = soup.find("div", class_ = "mejs__time mejs__duration-container")
print(video_duration_element.get_text())

03:06


In [19]:
# Convert the video duration to seconds
video_duration = video_duration_element.get_text()
minutes, seconds = map(int, video_duration.split(':'))
video_duration_seconds = minutes * 60 + seconds

print(video_duration_seconds)

# Specify filename WAV extension, as WAV files are uncompressed and preserve the original audio data
# This means they can provide better accuracy during the audio-to-text conversion process
audio_filename = "Observador_podcasts\\"+url.split('/')[-2]+(".wav")

SAMPLE_RATE = 48000              # [Hz]. sampling rate.

with sc.get_microphone(id=str(sc.default_speaker().name), include_loopback=True).recorder(samplerate=SAMPLE_RATE) as mic:
    # record audio with loopback from default speaker.
    data = mic.record(numframes=SAMPLE_RATE*video_duration_seconds)
    
    # change "data=data[:, 0]" to "data=data", if you would like to write audio as multiple-channels.
    sf.write(file=audio_filename, data=data[:, 0], samplerate=SAMPLE_RATE)

186
